# Gradient Descent using NumPy vs PyTorch

This is the 5th tutorial from Patrick Loeber's PyTorch tutorial playlist.

In [ ]:
import numpy as np

Let's create a linear regression model that has to predict the model as y = 2x.

In [ ]:
X = np.array([1, 2, 3, 4], dtype=np.float32)
Y = np.array([2, 4, 6, 8], dtype=np.float32)

w = 0.0

# model prediction
def forward(x):
  return w*x

# loss=MSE
def loss(y, y_predicted):
  return((y_predicted - y)**2).mean()

# Function --> 1/N (w*x - y_pred)**2
# Gradient --> 1/N 2*(w*x - y_pred)*x
def gradient(x, y, y_predicted):
  return np.dot(2*x, y_predicted-y).mean()

print(f'Prediction before training: f(5) = {forward(5):.3f}')

# Training
learning_rate = 0.01
n_iters = 10  # number of iterations

for epoch in range(n_iters):
  # prediction = forward pass
  y_pred = forward(X)

  # loss
  l = loss(Y, y_pred)

  # gradient
  dw = gradient(X, Y, y_pred)

  # update weights acc to gradient descent.
  w -= learning_rate * dw

  print(f'epoch {epoch+1}: w = {w:.3f}, loss = {l:.8f}')

print(f'Prediction after training: f(5) = {forward(5):.3f}')

Prediction before training: f(5) = 0.000
epoch 1: w = 1.200, loss = 30.00000000
epoch 2: w = 1.680, loss = 4.79999924
epoch 3: w = 1.872, loss = 0.76800019
epoch 4: w = 1.949, loss = 0.12288000
epoch 5: w = 1.980, loss = 0.01966083
epoch 6: w = 1.992, loss = 0.00314574
epoch 7: w = 1.997, loss = 0.00050331
epoch 8: w = 1.999, loss = 0.00008053
epoch 9: w = 1.999, loss = 0.00001288
epoch 10: w = 2.000, loss = 0.00000206
Prediction after training: f(5) = 9.999


Now, creating the same model but the gradient descent is calculated using PyTorch.

In [ ]:
import torch

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor([0.], dtype=torch.float32, requires_grad=True)

# model prediction
def forward(x):
  return w*x

# loss=MSE
def loss(y, y_predicted):
  return((y_predicted - y)**2).mean()

# This manual gradient function isn't needed now.

# the Tensor.item() function is always applied on a tensor that only has 1 value. This function returns the value of tensor as a scalar.
print(f'Prediction before training: f(5) = {forward(5).item():.3f}')  

# Training
learning_rate = 0.01
n_iters = 100  # number of iterations

for epoch in range(n_iters):
  # prediction = forward pass
  y_pred = forward(X)

  # loss
  l = loss(Y, y_pred)

  # gradient
  # dw = gradient(X, Y, y_pred)
  l.backward()

  # update weights acc to gradient descent.
  # Since the updation of weights should not be a part of gradient tracking, we have to use no_grad function here.
  with torch.no_grad():
    w -= learning_rate * w.grad

  # Set the gradients back to zero to prevent accumulation
  w.grad.zero_()

  if epoch%10 == 0:
    print(f'epoch {epoch+1}: w = {w.item():.3f}, loss = {l.item():.8f}')

print(f'Prediction after training: f(5) = {forward(5).item():.3f}')

Prediction before training: f(5) = 0.000
epoch 1: w = 0.300, loss = 30.00000000
epoch 11: w = 1.665, loss = 1.16278565
epoch 21: w = 1.934, loss = 0.04506890
epoch 31: w = 1.987, loss = 0.00174685
epoch 41: w = 1.997, loss = 0.00006770
epoch 51: w = 1.999, loss = 0.00000262
epoch 61: w = 2.000, loss = 0.00000010
epoch 71: w = 2.000, loss = 0.00000000
epoch 81: w = 2.000, loss = 0.00000000
epoch 91: w = 2.000, loss = 0.00000000
Prediction after training: f(5) = 10.000


Now, we're gonna do everything using PyTorch. So, we create the same model using PyTorch.

# Training Pipeline in PyTorch

We have 3 steps in this - 
1. Design our model (input size and output size, all the layers)
2. Construct the loss and the optimizer.
3. Training loop
- forward pass: Compute the gradient
- backward pass: Gradients
- update weights
- set gradients to zero
- repeat.

In [ ]:
import torch.nn as nn

X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32)

# w = torch.tensor([0.], dtype=torch.float32, requires_grad=True) Dont need this now. Model already knows.

n_samples, n_features = X.shape

# model prediction
input_size = n_features
output_size = n_features
model = nn.Linear(input_size, output_size)

# the Tensor.item() function is always applied on a tensor that only has 1 value. This function returns the value of tensor as a scalar.
print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

# Training
learning_rate = 0.06
n_iters = 250  # number of iterations

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # Stochastic Gradient Descent

for epoch in range(n_iters):
  # prediction = forward pass
  y_pred = model(X)

  # loss
  l = loss(Y, y_pred)

  # gradient
  # dw = gradient(X, Y, y_pred)
  l.backward()

  # update weights acc to gradient descent.
  optimizer.step()

  # Set the gradients back to zero to prevent accumulation
  optimizer.zero_grad()

  if epoch%10 == 0:
    [w, b] = model.parameters()
    print(f'epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {l.item():.8f}')

print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

Prediction before training: f(5) = -2.282
epoch 1: w = 1.836, loss = 46.52779388
epoch 11: w = 1.859, loss = 0.02960750
epoch 21: w = 1.883, loss = 0.02060417
epoch 31: w = 1.902, loss = 0.01433865
epoch 41: w = 1.918, loss = 0.00997842
epoch 51: w = 1.932, loss = 0.00694408
epoch 61: w = 1.943, loss = 0.00483245
epoch 71: w = 1.953, loss = 0.00336295
epoch 81: w = 1.960, loss = 0.00234031
epoch 91: w = 1.967, loss = 0.00162865
epoch 101: w = 1.972, loss = 0.00113340
epoch 111: w = 1.977, loss = 0.00078874
epoch 121: w = 1.981, loss = 0.00054890
epoch 131: w = 1.984, loss = 0.00038198
epoch 141: w = 1.987, loss = 0.00026582
epoch 151: w = 1.989, loss = 0.00018499
epoch 161: w = 1.991, loss = 0.00012874
epoch 171: w = 1.992, loss = 0.00008959
epoch 181: w = 1.994, loss = 0.00006235
epoch 191: w = 1.995, loss = 0.00004339
epoch 201: w = 1.996, loss = 0.00003019
epoch 211: w = 1.996, loss = 0.00002101
epoch 221: w = 1.997, loss = 0.00001462
epoch 231: w = 1.997, loss = 0.00001018
epoch 24

## Writing a custom Linear Regression Model in PyTorch.

In [ ]:
import torch
import torch.nn as nn

X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32)

# w = torch.tensor([0.], dtype=torch.float32, requires_grad=True) Dont need this now. Model already knows.

n_samples, n_features = X.shape

# model prediction
input_size = n_features
output_size = n_features

class LinearRegression(nn.Module):

  def __init__(self, input_dim, output_dim):
    # Initialize the super class.
    super(LinearRegression, self).__init__()
    
    # Define layers.
    self.lin = nn.Linear(input_dim, output_dim)

  def forward(self, x):
    return self.lin(x)

model = LinearRegression(input_size, output_size)


# the Tensor.item() function is always applied on a tensor that only has 1 value. This function returns the value of tensor as a scalar.
print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

# Training
learning_rate = 0.06
n_iters = 250  # number of iterations

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # Stochastic Gradient Descent

for epoch in range(n_iters):
  # prediction = forward pass
  y_pred = model(X)

  # loss
  l = loss(Y, y_pred)

  # gradient
  # dw = gradient(X, Y, y_pred)
  l.backward()

  # update weights acc to gradient descent.
  optimizer.step()

  # Set the gradients back to zero to prevent accumulation
  optimizer.zero_grad()

  if epoch%10 == 0:
    [w, b] = model.parameters()
    print(f'epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {l.item():.8f}')

print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')

Prediction before training: f(5) = 2.555
epoch 1: w = 1.729, loss = 15.53926945
epoch 11: w = 1.772, loss = 0.07775649
epoch 21: w = 1.810, loss = 0.05411150
epoch 31: w = 1.841, loss = 0.03765675
epoch 41: w = 1.868, loss = 0.02620573
epoch 51: w = 1.890, loss = 0.01823686
epoch 61: w = 1.908, loss = 0.01269120
epoch 71: w = 1.923, loss = 0.00883195
epoch 81: w = 1.936, loss = 0.00614624
epoch 91: w = 1.947, loss = 0.00427724
epoch 101: w = 1.955, loss = 0.00297656
epoch 111: w = 1.963, loss = 0.00207142
epoch 121: w = 1.969, loss = 0.00144153
epoch 131: w = 1.974, loss = 0.00100317
epoch 141: w = 1.978, loss = 0.00069812
epoch 151: w = 1.982, loss = 0.00048582
epoch 161: w = 1.985, loss = 0.00033809
epoch 171: w = 1.987, loss = 0.00023528
epoch 181: w = 1.990, loss = 0.00016374
epoch 191: w = 1.991, loss = 0.00011394
epoch 201: w = 1.993, loss = 0.00007930
epoch 211: w = 1.994, loss = 0.00005518
epoch 221: w = 1.995, loss = 0.00003840
epoch 231: w = 1.996, loss = 0.00002672
epoch 241